In [25]:
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import ast
import glob
import pingouin as pg

load_dotenv()

data_folder = os.getenv("DATA_FOLDER")

file_prefix = 'true_label_data'
file_prefix2 = 'label_data'

# Use glob to find all CSV files with the given filename start
file_list = glob.glob(f"{data_folder}/{file_prefix}*.csv")
file_list.extend(glob.glob(f"{data_folder}/{file_prefix2}*.csv"))

# Read each file into a DataFrame and concatenate them into a single DataFrame
dataframes = [pd.read_csv(file, sep=";",index_col=0) for file in file_list]
df = pd.concat(dataframes, ignore_index=True)

In [26]:
df

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,only_rel_90,Unrel_contain,Unrel_contain_65,Unrel_contain_70,avg_score,list_domains,list_scores,all_clean_urls,newsguard_dict,Labels
0,Day_1_4,R_2y8dRM3TUWjSPIo,Day_1,Misl,0,1,1,20,0,True,...,0.0,0.0,0.0,0.0,92.750000,"factcheck.org', 'thehill.com', 'newsweek.com',...","100,80,100,100,90,92.5,95,87.5,87.5,95",('https://www.factcheck.org/2021/07/debunking-...,"{'factcheck.org': 100, 'thehill.com': 80, 'new...","[{}, {}, {}, {'comparisons': [{'claim_article_..."
1,Day_1_4,R_bqJ3HoBP0lqi7CN,Day_1,True,1,5,3,30,0,True,...,0.0,0.0,0.0,0.0,94.642857,"recorder.maricopa.gov', ''', 'azcentral.com', ...","92.5,100,87.5,95,95,92.5,100",('https://recorder.maricopa.gov/electionresult...,{},"[{}, {}, {'comparisons': [{'claim_article_a': ..."
2,Day_1_4,R_5ovNt0RyjDe4zhD,Day_1,True,1,5,3,37,1,True,...,0.0,0.0,0.0,0.0,95.833333,"azcentral.com', 'abc15.com', 'thehill.com', 'p...","92.5,100,80,100,95,95,100,100,100",('https://www.azcentral.com/story/opinion/op-e...,"{'azcentral.com': 92.5, 'abc15.com': 100, 'the...","[{'comparisons': [{'claim_article_a': ""The lio..."
3,Day_1_4,R_1ILDrpPAAn0CdBx,Day_1,True,1,7,4,45,1,True,...,0.0,1.0,1.0,1.0,84.611111,"thehill.com', 'azcentral.com', 'abc15.com', 'w...","80,92.5,100,42,100,95,100,57,95",('https://thehill.com/homenews/campaign/563658...,"{'thehill.com': 80, 'azcentral.com': 92.5, 'ab...","[{}, {'comparisons': [{'claim_article_a': ""The..."
4,Day_1_4,R_3MycWKTJGEI3fkc,Day_1,True,1,7,4,44,1,True,...,0.0,0.0,0.0,0.0,95.833333,"azcentral.com', 'thehill.com', 'abc15.com', 'b...","92.5,80,100,95,100,95,100,100,100",('https://www.azcentral.com/story/opinion/op-e...,"{'azcentral.com': 92.5, 'thehill.com': 80, 'ab...","[{'comparisons': [{'claim_article_a': ""The lio..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,Day_9_3,R_3JqGYdN07AFVzOf,Day_9,Misl,0,1,1,41,0,FM,...,0.0,0.0,0.0,0.0,98.437500,"factcheck.org', 'yahoo.com', 'theintercept.com...","100,100,87.5,100,100,100,100,100",('https://www.factcheck.org/2021/10/scicheck-r...,"{'factcheck.org': 100, 'yahoo.com': 100, 'thei...","[{'comparisons': [{'claim_article_a': ""The NIH..."
1586,Day_9_3,R_1P6MQAwGMgs0Loo,Day_9,True,1,5,3,39,0,FM,...,0.0,0.0,0.0,0.0,94.166667,"googleadservices.com', 'scientificamerican.com...","100,77.5,87.5,100,100,100","('https://www.googleadservices.com/', 'https:/...","{'googleadservices.com': 100, 'scientificameri...","[{}, {'comparisons': [{'claim_article_a': ""The..."
1587,Day_9_3,R_rcgF4h6LylUXPUd,Day_9,Misl,0,1,2,23,0,FM,...,0.0,0.0,0.0,0.0,92.500000,"covid19.nih.gov', 'grants.nih.gov', 'grants.ni...","100,87.5,100,87.5,87.5",('https://covid19.nih.gov/funding#:~:text=NIH%...,"{'covid19.nih.gov': 100, 'grants.nih.gov': 87.5}","[{}, {}, {}, {}, {}, {'comparisons': [{'claim_..."
1588,Day_9_3,R_3JqGYdN07AFVzOf,Day_9,Misl,0,1,1,41,0,FM,...,0.0,1.0,1.0,1.0,85.777778,"science.org', 'the-scientist.com', 'rollcall.c...","100,100,75,82.5,82.5,95,80,57,100",('https://www.science.org/content/article/nih-...,"{'science.org': 100, 'the-scientist.com': 100,...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"


In [27]:
faulty_labels = 0
def safe_literal_eval(x):
    global faulty_labels
    try:
        return ast.literal_eval(x)
    except:
        faulty_labels += 1

df['Labels'] = df['Labels'].apply(safe_literal_eval)
print(faulty_labels)

101


In [28]:
df = df[df['Category'] != 'Coul']

In [29]:
df = df[df['Labels'].notna()]

# Feature Extraction

Features are: stance (headline+position; position); avg_score (Newsguard score); article_position

In [30]:
# taken from the feature_exploration notebook
def retrieve_features(label_dict_list, positions=False, headlines=False):
    #create three distinct lists to control each parameter
    # if label_dict_list is empty return NaN
    if not label_dict_list:
        return np.nan
    article_stances = []
    article_ng_scores = []
    article_serp_positions = []
    
    for article in label_dict_list: #this is the search level; here we can retrieve SERP position and Newsguard score
        if not article: #catch empty entries
            continue
        if len(article['comparisons']) == 0:
               continue
        #retrieve variables
        serp_newsguard = article['serp_newsguard']
        serp_position = article['serp_position']

        #add to lists for decomposed calculation
        article_ng_scores.append(serp_newsguard)
        article_serp_positions.append(serp_position)

        article_stance = 0
        for claim_pair in article['comparisons']: #this is the "article level"; here we can retrieve relative claim position and the bool headline info
            relation = claim_pair['relation']

            if relation == 0: #skip calculation logic for unrelated claims
                continue

            try:
                #retrieve relative claim positions
                fm_claim_position = claim_pair['fm_claim_position']
                serp_claim_position = claim_pair['serp_claim_position']

                #retrieve headline bool
                fm_claim_headline = claim_pair['fm_claim_headline']
                serp_claim_headline = claim_pair['serp_claim_headline']

            except:
                pass

            if not positions and not headlines: #calculate article stance only based on relation
                article_stance += relation
            if positions and not headlines: #calculate article stance based on relation and position
                if relation in [-1, 1]:
                    if serp_claim_position < fm_claim_position: #if serp claim earlier in the article than the respective fm claim
                        article_stance += relation + (serp_claim_position - fm_claim_position) if relation == -1 else relation - (serp_claim_position - fm_claim_position) #e.g. -1 - dif else 1 + dif (reinforcing in each direction)
                    else: # if serp claim later in article than the respective fm claim
                        article_stance += relation - (fm_claim_position - serp_claim_position) if relation == -1 else relation + (fm_claim_position - serp_claim_position) #e.g. - 1 + dif else 1 - dif (weakening in each direction)
            if positions and headlines:
                if relation in [-1, 1]:
                    individual_stance = 0
                    if serp_claim_position < fm_claim_position: #if serp claim earlier in the article than the respective fm claim
                        individual_stance = relation + (serp_claim_position - fm_claim_position) if relation == -1 else relation - (serp_claim_position - fm_claim_position) #e.g. -1 - dif else 1 + dif (reinforcing in each direction)
                        if serp_claim_headline:
                            individual_stance *= 2
                    else: # if serp claim later in article than the respective fm claim
                        individual_stance = relation - (fm_claim_position - serp_claim_position) if relation == -1 else relation + (fm_claim_position - serp_claim_position) #e.g. - 1 + dif else 1 - dif (weakening in each direction)
                        if serp_claim_headline:
                                individual_stance *= 2
                    article_stance += individual_stance
            if headlines and not positions:
                article_stance += relation * serp_claim_headline 
        if headlines and not positions:
            article_stances.append(article_stance/len(article['comparisons']))
        else:
            article_stances.append(article_stance/len(article['comparisons']))

    #if article_stances could not be successfully retrieved, return NaN
    if not article_stances:
        return np.nan
    
    article_ng_scores = np.array(article_ng_scores, dtype=float)
    # article_ng_scores = np.nan_to_num(article_ng_scores, nan=placeholder_ng_score) 

    return {'article_stances': article_stances, 'article_ng_scores': article_ng_scores, 'article_serp_positions': article_serp_positions}

In [31]:
df['Features_Raw']= df['Labels'].apply(retrieve_features)
df['Features_Pos']= df['Labels'].apply(lambda x: retrieve_features(x, positions=True, headlines=False))
df['Features_H']= df['Labels'].apply(lambda x: retrieve_features(x, positions=False, headlines=True))
df['Features_HP']= df['Labels'].apply(lambda x: retrieve_features(x, positions=True, headlines=True))

In [32]:
df['Features_Raw'][1]

{'article_stances': [-0.2, -0.12, 0.04, 0.0, -0.16],
 'article_ng_scores': array([nan, nan, nan, nan, nan]),
 'article_serp_positions': [3, 5, 6, 7, 8]}

In [33]:
#get the mean of all stances
def transform_stances(stances):
    return np.mean(stances)

#assign higher weight to articles higher up in the SERP
def transform_position(positions):
    positions = [1/pos for pos in positions]
    return np.mean(positions)

#transform the newsguard score into the range of 0 to 1
def transform_ng_score(ng_score):
    return ng_score/100

def transform_features(features, ng_score):
    if not isinstance(features, dict):
        return np.nan
    #extract features from feature dict
    stances = features['article_stances']
    positions = features['article_serp_positions']
    #transform stances
    stances = transform_stances(stances)
    #transform positions
    positions = transform_position(positions)
    #transform ng score
    ng_score = transform_ng_score(ng_score)

    features = [stances, positions, ng_score]
    if np.isnan(features).any():
        return np.nan
    return features


df['Features_Raw']= df.apply(lambda x: transform_features(x['Features_Raw'], x['avg_score']), axis=1)
df['Features_Pos']= df.apply(lambda x: transform_features(x['Features_Pos'], x['avg_score']), axis=1)
df['Features_H']= df.apply(lambda x: transform_features(x['Features_H'], x['avg_score']), axis=1)
df['Features_HP']= df.apply(lambda x: transform_features(x['Features_HP'], x['avg_score']), axis=1)

In [34]:
df['Features_Raw']

0       [-0.06666666666666667, 0.1640873015873016, 0.9...
1       [-0.08800000000000001, 0.19357142857142856, 0....
2       [0.015999999999999997, 0.3983333333333333, 0.9...
3          [0.05, 0.27777777777777773, 0.846111111111111]
4       [1.982541115402065e-18, 0.3168367346938776, 0....
                              ...                        
1585    [0.12000000000000002, 0.37666666666666665, 0.9...
1586    [-0.015999999999999997, 0.1957936507936508, 0....
1587     [0.14666666666666667, 0.1402116402116402, 0.925]
1588                                                  NaN
1589                                                  NaN
Name: Features_Raw, Length: 1357, dtype: object

In [35]:
label_mapping = {
    'Coul' : np.nan,
    'Misl' : 0,
    'True' : 1
}

df['User_Labels'] = df['Category'].map(label_mapping)

In [36]:
df = df.dropna(subset=['Features_Pos'])
df = df.dropna(subset=['User_Labels'])

In [37]:
df.groupby('User_Labels').count()

,Article_day,ResponseId,Day,Category,True_Dummy,Seven_Ordinal,Four_Ordinal,Age,Gender,FC_Eval,...,avg_score,list_domains,list_scores,all_clean_urls,newsguard_dict,Labels,Features_Raw,Features_Pos,Features_H,Features_HP
User_Labels,,,,,,,,,,,,,,,,,,,,,
0.0,374,374,374,374,374,374,374,374,374,374,...,374,374,374,374,374,374,374,374,374,374
1.0,692,692,692,692,692,692,692,692,692,692,...,692,692,692,692,692,692,692,692,692,692


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut, StratifiedKFold

df_clean = df.dropna(subset=['Features_Pos', 'User_Labels'])

X = np.array(df['Features_Pos'].tolist())
y = np.array(df['User_Labels'].tolist())

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1860)
#loo = LeaveOneOut()

model = LogisticRegression(class_weight='balanced')

# Compute evaluation metrics
accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy') 
precision = cross_val_score(model, X, y, cv=skf, scoring='precision') 
recall = cross_val_score(model, X, y, cv=skf, scoring='recall') 
f1 = cross_val_score(model, X, y, cv=skf, scoring='f1') 

# Display mean results
print(f"Accuracy: {np.mean(accuracy):.3f}")
print(f"Precision: {np.mean(precision):.3f}")
print(f"Recall: {np.mean(recall):.3f}")
print(f"F1-score: {np.mean(f1):.3f}")

Accuracy: 0.638
Precision: 0.781
Recall: 0.617
F1-score: 0.687
